## Install uncommon packages (will supply requements.txt later)

In [217]:
#!pip install git+https://github.com/boudinfl/pke.git
#!pip install wordninja
#!pip install pyenchant

## ijson for macos:

In [ ]:
#brew install cmake
#git clone git@github.com:lloyd/yajl.git
#cd yajl
#./configure && make install

In [14]:
import pandas as pd
import numpy as np
import os
import json
import wordninja
from os import listdir
import re
import sys
import ast
import pke
import enchant
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import requests
import pickle
import datetime
import ijson.backends.yajl2_cffi as ijson
from geopy.geocoders import Bing
from geopy.extra.rate_limiter import RateLimiter
from urlextract import URLExtract



nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
d = enchant.Dict("en_US") # ENGLISH Dictionary


# CHANGE THE PATH TO A DATASET HERE
basedir = os.path.join(os.sep, "media", "tie-server", "DATA", "Jens", "Crunchbase")
#tempdir = os.getcwd()
tempdir = basedir
path_to_embedrank_repo = os.path.join("ai-research-keyphrase-extraction") # relative (without "../") or absolute path

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tie-server/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/tie-server/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tie-
[nltk_data]     server/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to /home/tie-
[nltk_data]     server/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/tie-server/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [117]:
data = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organization_descriptions.csv"), index_col="uuid")
#data.iloc[:,1].to_csv('CB_Export_17_08_07/descr.txt')
print("loaded descriptions")
data1 = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organizations.csv"), low_memory=False, index_col="uuid")
print("loaded org infos")

loaded descriptions
loaded org infos


In [123]:
# Merge 2 csv files
result = pd.merge(data, data1, how='outer', left_index=True, right_index=True)
result1 = pd.merge(data, data1, how='inner', left_index=True, right_index=True)
print("merged")
# Delete all companies that do not have any descriptions
result1 = result1.dropna(subset=["description"])

merged


In [124]:
print(((result.shape[0] - result1.shape[0])/result.shape[0]*100), "% of companies do not have any description")

33.484516587151056 % of companies do not have any description


In [125]:
result1.head()

,description,company_name,primary_role,permalink,domain,homepage_url,country_code,state_code,region,city,...,phone,facebook_url,linkedin_url,cb_url,logo_url,profile_image_url,twitter_url,alias,created_at,updated_at
uuid,,,,,,,,,,,,,,,,,,,,,
f5bb580f-d655-cf3f-9ade-eca5b3f2719f,"Android, Apple - iOS, Blackberry, Windows Phon...",VilarikA,company,/organization/vilarika,vilarika.com.br,http://vilarika.com.br/,BRA,NaN,Rio de Janeiro,Belo Horizonte,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/vilarika,https://www.crunchbase.com/organization/vilari...,http://public.crunchbase.com/t_api_images/v141...,NaN,NaN,2014-12-11 06:46:05,2016-09-07 00:03:51.67913
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...,Formel D GmbH,company,/organization/formel-d-gmbh,formeld.com,http://www.formeld.com,DEU,NaN,DEU - Other,Troisdorf,...,+49 2241 9960,https://www.facebook.com/formeld,https://www.linkedin.com/company/formel-d-group,https://www.crunchbase.com/organization/formel...,https://www.crunchbase.com/organization/formel...,http://public.crunchbase.com/t_api_images/v148...,https://www.twitter.com/formeld_es,NaN,2016-06-01 06:58:37.692725,2017-07-18 07:22:19.15864
f5bd38d1-7719-2935-fb6c-defae39f5b93,[iNFoGooL](http://infogool.com) - The Informat...,infogool,company,/organization/infogool,infogool.com,http://infogool.com,NaN,NaN,NaN,NaN,...,8802393009,http://www.facebook.com/infogool,NaN,https://www.crunchbase.com/organization/infogool,https://www.crunchbase.com/organization/infogo...,http://public.crunchbase.com/t_api_images/v139...,https://www.twitter.com/infogool,NaN,2014-04-12 00:10:12,2016-09-08 22:02:05.585875
773adc18-132a-937d-8841-4f833e64dd56,The Jinfeng Gold Mine is an combined open pit ...,Jinfeng Mine,company,/organization/jinfeng-mine,NaN,NaN,CHN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/jinfen...,NaN,NaN,NaN,NaN,2016-12-07 06:43:06.955155,2016-12-07 06:45:09.14727
f5bdd48a-a09f-2f4c-bd71-4c4207c7731b,Peardoc offers online tools to convert HTML to...,Peardoc Solutions,company,/organization/peardoc-solutions,peardoc.com,http://www.peardoc.com,IND,NaN,Chennai,Chennai,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/peardo...,https://www.crunchbase.com/organization/peardo...,http://public.crunchbase.com/t_api_images/v140...,https://www.twitter.com/pear_doc,NaN,2014-07-17 10:06:03,2016-03-08 02:56:01.230586


In [128]:
print("Total amount of companies with descriptions: " + str(len(result1)))

Total amount of companies with descriptions: 353146


In [144]:
# Construct a list of companies from dataset folder
f_companies_list = os.listdir(os.path.join(basedir, "News_1"))
# Get rid of ".json" extension and apply uppercase
f_companies_list = [re.sub(".json", "", company.lower()) for company in f_companies_list]
print(len(f_companies_list))

20247


In [147]:
# Prepare the data frame of companies descriptions to be used in final data frame with articles
d_companies_df = result1
d_companies_df.loc[:,'company_name'] = d_companies_df.loc[:,'company_name'].str.lower()
#d_companies_df = d_companies_df[d_companies_df['company_name'].str.lower().isin(d_companies_list)]
#d_companies_df = d_companies_df[d_companies_df['company_name'].isin(d_companies_list)]
d_companies_df = d_companies_df.reset_index().set_index(keys='company_name')#, verify_integrity=True)



# Delete the companies which have the same name
d_companies_df = d_companies_df[~d_companies_df.index.duplicated(keep=False)]


d_companies_df = d_companies_df[['uuid', 'description']]
len(d_companies_df)

345890

In [148]:
# Delete companies outside or dataset of articles
companies_df = d_companies_df[d_companies_df.index.isin(f_companies_list)]
companies_list = companies_df.index.values
print(len(companies_df))

10931


In [149]:
# Get rid of companies with names containing anything else than latin letters (as well as containing "_" separating multiple words in names)
companies_list = [company.lower() for company in companies_list if all(letter in string.ascii_letters for letter in company)]
print(len(companies_list))

# Get rid of companies whose full names consist of one single word from english dictionary
companies_list = [company for company in companies_list if not d.check(company.lower())]
print(len(companies_list))

10684
9936


In [ ]:
def word_exists(word, apiKey=apiKey, apiUrl='http://api.wordnik.com/v4'):

    # Construct the API URL for a random-word query
    api_url = "{baseurl}/words.json/search/{word}".format(
        baseurl=apiUrl,
        word=word
    )

    parameters = {
        'api_key': apiKey,
    }

    # Perform the query and store the HTTP response object
    response = requests.get(api_url, params=parameters)

    # Convert the response content to a list
    # NOTE: The content is initially returned as a byte string
    word_object = json.loads(response.content)

    # Get the number word's instances in the dictionary
    if word_object:
        if word_object['searchResults'][0]['count'] > 0:
            result = True
        else:
            result = False
    else:
        result = False

    # Return the word as a string
    return result

In [125]:
# Same as the previous step (looking for english words) but with Wordnik API
apiUrl = 'http://api.wordnik.com/v4'
apiKey = 'b0a856e4cbe5c4e3f110d0527b901b7064f22a1c09e153547'
# Extract the list words from Wordnik that DO exist
wordnik_companies_list = [company for company in companies_list if word_exists(company)]
print(len(wordnik_companies_list))

# These words DO exist in Wordnik dictionary
print((wordnik_companies_list))

['accend', 'acton', 'aerospike', 'afterschool', 'ais', 'alation', 'alector', 'alignable', 'amberjack', 'amicus', 'ananas', 'ansible', 'anyroad', 'aptera', 'aruspex', 'asana', 'attask', 'auris', 'authorly', 'aviso', 'ayr', 'backchat', 'backdoor', 'backtrace', 'bannerman', 'bastille', 'beme', 'bento', 'bevvy', 'biome', 'biscotti', 'bittorrent', 'blesh', 'bonafide', 'bonobos', 'bookbag', 'boombox', 'booyah', 'bottlenose', 'bowery', 'boxfish', 'brit', 'browster', 'carbonite', 'cardio', 'cartesian', 'cartogram', 'celly', 'centro', 'chargeback', 'chockstone', 'chronicity', 'clickable', 'cloze', 'containership', 'contentful', 'convo', 'copromote', 'courseload', 'courseloads', 'cringle', 'crossfader', 'cuculus', 'dekko', 'dextro', 'digby', 'doo', 'doozie', 'doppelganger', 'driverside', 'dropoff', 'dropship', 'duetto', 'eco', 'elastica', 'elsen', 'emma', 'endorphin', 'endplay', 'enow', 'entrada', 'epiphyte', 'eponym', 'eps', 'ess', 'everly', 'evolv', 'evolver', 'exogenesis', 'eyespot', 'fab', '

In [ ]:
# Save for later use 
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
with open(os.path.join(tempdir, 'temp_data','wordnik_companies_list.txt'), 'wb') as fp:
    pickle.dump(wordnik_companies_list, fp)

In [150]:
with open (os.path.join(tempdir, 'temp_data', 'wordnik_companies_list.txt'), 'rb') as fp:
    wordnik_companies_list = pickle.load(fp)

In [133]:
# Too many false positives
# Tries to separate a name string into multiple words and if succeeds gets rid of the company (a lot of false positives)
# companies_list = [company for company in companies_list if len(wordninja.split(company.lower())) < 2]
# print(len(companies_list))

In [151]:
companies_list = [e for e in companies_list if e not in set(wordnik_companies_list)]
print(len(companies_list))
print(companies_list[0:100])

9658
['nerites', 'biovascular', 'transbiotec', 'thrombovision', 'fortressware', 'songfor', 'buildpulse', 'gogograndparent', 'changenexus', 'connectbeam', 'connoshoer', 'multispan', 'oculusit', 'loctote', 'freewavz', 'fabtask', 'fantrail', 'krauttools', 'prenova', 'freepath', 'naturtrip', 'kabongo', 'crystalplex', 'bindhq', 'phonio', 'invup', 'zynbit', 'statementone', 'neighbormd', 'xbux', 'helicomm', 'arrister', 'remotereps', 'artvenue', 'doblet', 'duxplore', 'sereniti', 'soligence', 'ecinity', 'coeio', 'sypherlink', 'favim', 'aktino', 'streammosaic', 'careparent', 'xmybox', 'wawadoo', 'inveshare', 'vicino', 'fieldaware', 'movio', 'sedline', 'blabfeed', 'adnoviv', 'sproutster', 'lumetrics', 'berggi', 'adolade', 'golorry', 'tempmine', 'curely', 'cloudstitch', 'recotech', 'sparkwords', 'bollyx', 'stitzii', 'fastpoint', 'hitbills', 'ironqloud', 'spiralcat', 'vaporslide', 'worklete', 'empowrnet', 'neuropure', 'cultur', 'aqs', 'dittit', 'dispensesource', 'proxilliant', 'zonder', 'jodange', 

In [152]:
# Delete the companies that have the same names as geographical entities

geolocator = Bing(api_key="AhUn6KYuWon2vOPYrpEMThB3Z7RqsXvcXjbLK_svjSn0Uk7lN5mG23Z7nJj_3fjF", timeout=5)
geocode = RateLimiter(geolocator.geocode)

geo_companies_list = []

print("List of companies that have the same names as geographical entities:")
for i, company in enumerate(companies_list):
    location = geocode(company)
    if location:
        geo_companies_list.append(company)
        print(company)


List of companies that have the same names as geographical entities:
nerites
kabongo
xbux
helicomm
remotereps
streammosaic
vicino
adolade
vaporslide
zonder
applilog
abom
gradschoolloans
screenmedix
ammasso
nesteggguru
mailexpress
sarvamail
buildmymove
tutton
onlinesheetmusic
simplemist
haywheel
stellaris
flynn
parentplus
elematics
allihub
avinti
mygogames
jomaja
jamanimal
arav
radmit
snowdon
mangia
diffon
marketrange
hirexperience
groupstream
latherm
brandshub
taperecorder
mentordotme
ivrtrain
ketra
periscape
excalibur
caterva
bluespotpark
pinkberry
zuki
nutricate
unum
symphire
mpv
nanocompound
globbux
geneexcel
skipola
abaris
smartstay
weissenhaus
marketinstant
priceadvice
dowhistle
koniku
rehabdev
oomi
pinkdingo
describeme
heartscan
monki
budz
sketchme
oscar
zillow
sermo
agari
reachlocal
custora
dstillery
jash
angellist
domo
sprinklr
bebo
panaya
roku
rdio
turo
orderup
kony
clairmail
vts
seworks
cylance
altschool
maana
ooma
soraa
datameer
startupdigest
squarefoot
viki
viglink
andela
s

legaljump
piauto
startplatz
bluesprig
shopjester
vsporto
stukent
igor
trackin
cuisinelinks
weatherista
dotheglobe
fashionreverie
skybulls
inmediata
sweetist
roomi
sondors
onedrop
nexdentist
octavian
ethera
caeden
connectem
mieple
zaza
evalu
nuelle
netminions
besos
loogla
geekstatus
vestly
videocodes
dormnoise
imara
clearmedicare
hypepoints
krowder
narvii
bueeno
trafficmac
krak
rc
purewine
makeena
dineintime
tragara
cequint
liveon
lilluxe
cosnet
datesalad
youarrive
ofuz
stryd
roombeats


In [153]:
# Save for later use 
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
with open(os.path.join(tempdir, 'temp_data','geo_companies_list.txt'), 'wb') as fp:
    pickle.dump(geo_companies_list, fp)

In [154]:
with open (os.path.join(tempdir, 'temp_data', 'geo_companies_list.txt'), 'rb') as fp:
    geo_companies_list = pickle.load(fp)

In [155]:
# Delete the companies from geo list
companies_list = [e for e in companies_list if e not in set(geo_companies_list)]
print(len(companies_list))

8618


In [158]:
# Create df with descriptions
companies_df = d_companies_df[d_companies_df.index.isin(companies_list)]
print(len(companies_df))

8618


In [159]:
# Save for later use with keywords extraction
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
with open(os.path.join(tempdir, 'temp_data', 'companies_df'), 'wb') as fp:
    pickle.dump(companies_df, fp)

In [9]:
with open (os.path.join(tempdir, 'temp_data', 'companies_df'), 'rb') as fp:
    companies_df = pickle.load(fp)
companies_list = companies_df.index.values
print(len(companies_df))
companies_df.head()


8618


,uuid,description
company_name,,
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur..."
songfor,7789da2f-f1dd-a1ac-434b-355c320b4d71,Auf SongFor.com hast du die Möglichkeit Musik ...


In [32]:
### TWO WAYS TO EXTRACT KEYWORDS ###




# With PKE Package - TopicRank
# initialize keyphrase extraction model, here TopicRank
import pke

# With EmbedRank
cwd = os.getcwd()
try:
    os.chdir(os.path.join(os.path.dirname(cwd), path_to_embedrank_repo))
except FileNotFoundError:
    try:
        os.chdir(path_to_embedrank_repo)
    except FileNotFoundError:
        os.chdir(cwd) 
import launch
print(os.getcwd())
embedding_distributor = launch.load_local_embedding_distributor('en')
pos_tagger = launch.load_local_pos_tagger('en')

/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction


/home/tie-server/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [203]:
k_companies_df.head()

,uuid,description,kp_topicrank,kp_embedrank
company_name,,,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog...","[market, technical innovation, company, synthe...","[world class technology, natural tissue repair..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[classâ€ products, hemodialysis access graft,...","[human clinical trials, hemodialysis access gr..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[systems, development stage company, company, ...","[blood alcohol detection system, construction ..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...,"[thrombovision, medical diagnosis, biomedical ...","[medical diagnosis, critical cardiovascular ri..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[access policies, fortress capsule, manages, s...","[access policies, fortress capsule, virtual wo..."


In [ ]:
counter = 0
k_companies_df = companies_df.copy()

for index, row in companies_df.iterrows():
    
    print(str(datetime.datetime.now()) + " " + index)    
    
    counter +=1
    #if counter >3:
    #    break
    
    extractor = pke.unsupervised.TopicRank()
    # load the content of the document, here document is expected to be in raw
    # format (i.e. a simple string of text) and preprocessing is carried out using nltk
    s = row["description"]
    extractor.read_text(s)

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives
    extractor.candidate_selection()

    try:
        # candidate weighting, in the case of TopicRank: using a random walk algorithm
        extractor.candidate_weighting()
        # N-best selection, keyphrases contains the 10 highest scored candidates as
        # (keyphrase, score) tuples
        keyphrases = extractor.get_n_best(n=5, stemming=False)
        keyphrases = sorted(keyphrases, key=lambda x: x[1], reverse=True)

        k_companies_df.loc[index, 'kp_topicrank'] = str(keyphrases)
    
    except ValueError:
        print("TOPIC_RANK: Description for company " + index + 
              " (" + row['uuid'] + ")" +
              " is too short. Description: " + index)
        continue
    
    kp1 = launch.extract_keyphrases(embedding_distributor, pos_tagger, s, 5, 'en') 
    if None in kp1:
        print("EMBED_RANK: Description for company " + index + 
                  " (" + row['uuid'] + ")" +
                  " is too short. Description: " + index)
        continue    
    k_companies_df.loc[index, 'kp_embedrank'] = str(kp1)
    
k_companies_df.dropna(subset=["kp_topicrank"], inplace=True)
k_companies_df.dropna(subset=["kp_embedrank"], inplace=True)
os.chdir(cwd)

2018-07-23 21:29:15.836455nerites
2018-07-23 21:29:18.243740biovascular
2018-07-23 21:29:20.784224transbiotec
2018-07-23 21:29:23.402018thrombovision
2018-07-23 21:29:25.952495fortressware
2018-07-23 21:29:28.504126songfor
2018-07-23 21:29:31.250156buildpulse
2018-07-23 21:29:33.834544gogograndparent
2018-07-23 21:29:36.369559changenexus


/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:44: RuntimeWarning: invalid value encountered in true_divide
  0.5 + (sim_between_norm - np.nanmean(sim_between_norm, axis=0)) / np.nanstd(sim_between_norm, axis=0)


2018-07-23 21:29:39.063988connectbeam
2018-07-23 21:29:41.654636connoshoer
2018-07-23 21:29:44.185940multispan
2018-07-23 21:29:46.750198oculusit
2018-07-23 21:29:49.370381loctote
2018-07-23 21:29:52.036354freewavz
2018-07-23 21:29:54.547487fabtask
2018-07-23 21:29:57.189848fantrail
2018-07-23 21:29:59.740534krauttools
2018-07-23 21:30:02.293665prenova
2018-07-23 21:30:04.920419freepath
2018-07-23 21:30:07.409902naturtrip
2018-07-23 21:30:09.895554kabongo
2018-07-23 21:30:12.374737crystalplex
2018-07-23 21:30:14.973391bindhq
2018-07-23 21:30:17.559213phonio
2018-07-23 21:30:20.232466invup
2018-07-23 21:30:22.750827zynbit
2018-07-23 21:30:25.477715statementone
2018-07-23 21:30:28.001336neighbormd
2018-07-23 21:30:30.591867xbux
2018-07-23 21:30:33.145078helicomm
2018-07-23 21:30:35.717279arrister
2018-07-23 21:30:38.251769remotereps
2018-07-23 21:30:40.843788artvenue
2018-07-23 21:30:43.321783doblet
2018-07-23 21:30:45.849859duxplore
2018-07-23 21:30:48.558664sereniti
2018-07-23 21:30:51

/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:37: RuntimeWarning: invalid value encountered in true_divide
  doc_sim_norm = 0.5 + (doc_sim_norm - np.average(doc_sim_norm)) / np.std(doc_sim_norm)
/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:42: RuntimeWarning: All-NaN slice encountered
  sim_between_norm = sim_between/np.nanmax(sim_between, axis=0)
/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:44: RuntimeWarning: Mean of empty slice
  0.5 + (sim_between_norm - np.nanmean(sim_between_norm, axis=0)) / np.nanstd(sim_between_norm, axis=0)
/home/tie-server/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1427: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


2018-07-23 21:33:51.036289eainvest
2018-07-23 21:33:53.537516cleanfish
2018-07-23 21:33:56.217629preisbock
2018-07-23 21:33:58.824156caserev
2018-07-23 21:34:01.394542screenmedix
2018-07-23 21:34:03.942409primedic
2018-07-23 21:34:06.501635treatful
2018-07-23 21:34:09.068809calyxo
2018-07-23 21:34:11.767354workmob
2018-07-23 21:34:14.389074globein
2018-07-23 21:34:16.926770ammasso
2018-07-23 21:34:19.514855empliant
2018-07-23 21:34:22.127885nesteggguru
2018-07-23 21:34:24.674372aerobo
2018-07-23 21:34:27.215945fortiusone
2018-07-23 21:34:29.718377gigacrete
2018-07-23 21:34:32.309962alverix
2018-07-23 21:34:34.829370informous
2018-07-23 21:34:37.315183inviragen
2018-07-23 21:34:39.886843polimetrix
2018-07-23 21:34:42.632215proteopure
2018-07-23 21:34:45.097752ocdesk
2018-07-23 21:34:47.800056biotrackthc
2018-07-23 21:34:50.420718angiogenex
2018-07-23 21:34:52.930081hyperpot
2018-07-23 21:34:55.497267siut
2018-07-23 21:34:58.092857gonoogie
2018-07-23 21:35:00.586139mailexpress
2018-07-23

...

2018-07-24 02:00:34.860058solmentum
2018-07-24 02:00:37.545885goodworld
2018-07-24 02:00:40.111066sampa
2018-07-24 02:00:42.844322havemyshift
2018-07-24 02:00:45.570425minitime
2018-07-24 02:00:48.161426jamplify
2018-07-24 02:00:50.726898judicata
2018-07-24 02:00:53.261623homeunion
2018-07-24 02:00:55.739679mindmixer
2018-07-24 02:00:58.396594brightlot
2018-07-24 02:01:00.913320codersclan
2018-07-24 02:01:03.487789doobop
2018-07-24 02:01:06.193499kustomnote
2018-07-24 02:01:08.698418cirrosecure
2018-07-24 02:01:11.259025murfie
2018-07-24 02:01:13.794549prepflash
2018-07-24 02:01:16.328620prepify
2018-07-24 02:01:18.824248callsfreecalls
2018-07-24 02:01:21.471230feedtrace
2018-07-24 02:01:24.020435elucify
2018-07-24 02:01:26.557895pilldrops
2018-07-24 02:01:29.267435clinicast
2018-07-24 02:01:31.973760retrotope
2018-07-24 02:01:34.494797marxent
2018-07-24 02:01:37.083687lumenergi
2018-07-24 02:01:39.687373wellinks
2018-07-24 02:01:42.224713bunndle
2018-07-24 02:01:44.839447mdxhealth
201

2018-07-24 02:10:43.042309uavionix
2018-07-24 02:10:45.617074viraltag
2018-07-24 02:10:48.234935qualvu
2018-07-24 02:10:50.850845sickweather
2018-07-24 02:10:53.343755frs
2018-07-24 02:10:56.146787teraki
2018-07-24 02:10:58.661613bolstra
2018-07-24 02:11:01.279780eyenalyze
2018-07-24 02:11:04.033334triplebyte
2018-07-24 02:11:06.514086datacamp
2018-07-24 02:11:09.066301fixetude
2018-07-24 02:11:11.373752qwil
2018-07-24 02:11:13.931974pacatio
2018-07-24 02:11:16.486468nousdecor
2018-07-24 02:11:19.116640plumtv
2018-07-24 02:11:21.605633clearflow
2018-07-24 02:11:24.334660stylezen
2018-07-24 02:11:26.927112comvibe
2018-07-24 02:11:29.571593stayful
2018-07-24 02:11:32.108694bellabeat
2018-07-24 02:11:34.652054readfy
2018-07-24 02:11:37.386248wehostels
2018-07-24 02:11:39.912684deskgod
2018-07-24 02:11:43.132466brandshield
2018-07-24 02:11:45.927842evogen
2018-07-24 02:11:48.489782madefire
2018-07-24 02:11:51.101987remesh
2018-07-24 02:11:53.601915lawpal
2018-07-24 02:11:56.134658gopuff
20

2018-07-24 02:20:53.392470umake
2018-07-24 02:20:55.924474prosky
2018-07-24 02:20:58.441400rivs
2018-07-24 02:21:00.928527engrade
2018-07-24 02:21:03.490585readz
2018-07-24 02:21:06.022569greenphire
2018-07-24 02:21:08.592704colabo
2018-07-24 02:21:11.150027iggbo
2018-07-24 02:21:13.763169viewlift
2018-07-24 02:21:16.465409trovix
2018-07-24 02:21:19.254606courbanize
2018-07-24 02:21:21.487818focalcast
2018-07-24 02:21:24.025851lookcast
2018-07-24 02:21:26.623597curtran
2018-07-24 02:21:29.200670gdgt
2018-07-24 02:21:31.898322crashlytics
2018-07-24 02:21:34.477959keywee
2018-07-24 02:21:37.083105powerlytics
2018-07-24 02:21:39.598405textpower
2018-07-24 02:21:42.267583tipbit
2018-07-24 02:21:44.777136fattmerchant
2018-07-24 02:21:47.413122nascentric
2018-07-24 02:21:49.975684visarity
2018-07-24 02:21:52.520824neoantigenics
2018-07-24 02:21:55.112168synerchip
2018-07-24 02:21:57.765381didlog
2018-07-24 02:22:00.361924ecomdash
2018-07-24 02:22:02.888350fhoosh
2018-07-24 02:22:05.491826yep

2018-07-24 02:30:33.808511snappydata
2018-07-24 02:30:36.318374useready
2018-07-24 02:30:38.968675frontstream
2018-07-24 02:30:41.573506foneshow
2018-07-24 02:30:44.195271beegit
2018-07-24 02:30:46.812633naverus
2018-07-24 02:30:49.444845vaycayhero
2018-07-24 02:30:52.019627avogy
2018-07-24 02:30:54.537674vestwell
2018-07-24 02:30:57.064442eagerpanda
2018-07-24 02:30:59.704715sipsynergy
2018-07-24 02:31:02.244111eyefluence
2018-07-24 02:31:04.930046actualmeds
2018-07-24 02:31:07.565377nestiny
2018-07-24 02:31:10.185717showkit
2018-07-24 02:31:12.910725songcat
2018-07-24 02:31:15.431286urtak
2018-07-24 02:31:18.005078labnow
2018-07-24 02:31:20.636423noovo
2018-07-24 02:31:23.200935akitabox
2018-07-24 02:31:25.711845crexi
2018-07-24 02:31:28.294609oomnitza
2018-07-24 02:31:30.936039emms
TOPIC_RANK: Description for company emms (4eeacc81-59cd-5d60-e32b-313d6d0e9d9a) is too short. Description: emms
2018-07-24 02:31:30.939340healbe
2018-07-24 02:31:33.661989fabbeo
2018-07-24 02:31:36.360975

2018-07-24 02:40:37.080394flowcardia
2018-07-24 02:40:39.718202peachworks
2018-07-24 02:40:42.298971paydivvy
2018-07-24 02:40:44.981314kitchenbowl
2018-07-24 02:40:47.500778moodnode
2018-07-24 02:40:50.127344repairogen
2018-07-24 02:40:52.803433youtern
2018-07-24 02:40:55.521505homewellness
2018-07-24 02:40:58.170008auctiondrop
2018-07-24 02:41:00.743833xtent
2018-07-24 02:41:03.295275diy
2018-07-24 02:41:05.970071watchsend
2018-07-24 02:41:08.463869shotpoint
2018-07-24 02:41:11.006017futureleague
2018-07-24 02:41:13.579879intronet
2018-07-24 02:41:16.233641moneythink
2018-07-24 02:41:18.757143triplingo
2018-07-24 02:41:21.498256bindo
2018-07-24 02:41:24.035827cardtapp
2018-07-24 02:41:26.599885tivity
2018-07-24 02:41:29.156892zeepro
2018-07-24 02:41:31.599152perceivant
2018-07-24 02:41:34.089237soley
2018-07-24 02:41:36.404635levlr
2018-07-24 02:41:38.845804pacinian
2018-07-24 02:41:41.520683ampsy
2018-07-24 02:41:44.211396smartfile
2018-07-24 02:41:46.959300accelera
2018-07-24 02:41:

In [37]:
print(str(len(companies_df) - len(k_companies_df)) + " companies did not have sufficient length of descriptions")
k_companies_df.tail()

23 companies did not have sufficient length of descriptions


,uuid,description
company_name,,
hooja,ff71401a-f5b0-1dd8-2b72-4a9e5d6ecba0,Hooja is a company focused on the mobile realm...
baseclick,d140620f-09bb-dd33-baaa-ecffff1d8bc6,baseclick GmbH is a start-up company funded an...
biodetego,fef487a3-2505-e176-7bdc-d82db18525dc,"BioDetego is developing, VASPfore, a new bioma..."
socialthreader,ffe12892-05c4-e8e4-d728-d9b54e7b3aeb,SocialThreader increases effectiveness of digi...
findally,cc693948-abf3-e3dc-f845-3e38ac718aab,Findally is an image-based product engine that...


In [4]:
k_with_confidences_companies_df = k_companies_df.copy()

In [5]:
counter = 0
for company, row in k_with_confidences_companies_df.iterrows():
    counter = counter + 1
    #if counter > 2:
    #    break
    #print(str(datetime.datetime.now()) + " " + company)
    k_companies_df.loc[company,"kp_embedrank"] = ast.literal_eval(row["kp_embedrank"])[0]
    k_companies_df.loc[company,"kp_topicrank"] = [kp[0] for kp in ast.literal_eval(row["kp_topicrank"])]
k_companies_df.head()
#print(counter)

2018-08-05 21:13:04.028434 nerites
2018-08-05 21:13:04.076119 biovascular
2018-08-05 21:13:04.081710 transbiotec
2018-08-05 21:13:04.087256 thrombovision
2018-08-05 21:13:04.092811 fortressware
2018-08-05 21:13:04.098354 songfor
2018-08-05 21:13:04.103898 buildpulse
2018-08-05 21:13:04.109405 gogograndparent
2018-08-05 21:13:04.114926 changenexus
2018-08-05 21:13:04.120409 connectbeam
2018-08-05 21:13:04.125943 connoshoer
2018-08-05 21:13:04.131451 multispan
2018-08-05 21:13:04.136959 oculusit
2018-08-05 21:13:04.142497 loctote
2018-08-05 21:13:04.147998 freewavz
2018-08-05 21:13:04.153507 fabtask
2018-08-05 21:13:04.159022 fantrail
2018-08-05 21:13:04.164528 krauttools
2018-08-05 21:13:04.170011 prenova
2018-08-05 21:13:04.175692 freepath
2018-08-05 21:13:04.181199 naturtrip
2018-08-05 21:13:04.188198 kabongo
2018-08-05 21:13:04.193695 crystalplex
2018-08-05 21:13:04.199227 bindhq
2018-08-05 21:13:04.209866 phonio
2018-08-05 21:13:04.215372 invup
2018-08-05 21:13:04.220876 zynbit
2018

2018-08-05 21:13:05.916876 actionality
2018-08-05 21:13:05.923246 nanopack
2018-08-05 21:13:05.928737 provasculon
2018-08-05 21:13:05.934237 therative
2018-08-05 21:13:05.939734 fatpipe
2018-08-05 21:13:05.945292 ipawn
2018-08-05 21:13:05.951052 aitainment
2018-08-05 21:13:05.956547 mentinova
2018-08-05 21:13:05.962008 taperecorder
2018-08-05 21:13:05.967504 envirokure
2018-08-05 21:13:05.972988 skyphrase
2018-08-05 21:13:05.978518 mentordotme
2018-08-05 21:13:05.983966 telkore
2018-08-05 21:13:05.989454 beekly
2018-08-05 21:13:05.994960 synfora
2018-08-05 21:13:06.000446 ivrtrain
2018-08-05 21:13:06.006360 scholarpro
2018-08-05 21:13:06.011842 runiq
2018-08-05 21:13:06.017338 vioso
2018-08-05 21:13:06.022941 ketra
2018-08-05 21:13:06.028453 skaphandrus
2018-08-05 21:13:06.033986 attachstor
2018-08-05 21:13:06.039523 instrumentlife
2018-08-05 21:13:06.045059 rfeyed
2018-08-05 21:13:06.050627 periscape
2018-08-05 21:13:06.056166 prazas
2018-08-05 21:13:06.061732 kolorific
2018-08-05 21:

2018-08-05 21:13:07.899886 dialpad
2018-08-05 21:13:07.905538 foodpanda
2018-08-05 21:13:07.911045 wimdu
2018-08-05 21:13:07.916540 angellist
2018-08-05 21:13:07.922038 webvisible
2018-08-05 21:13:07.927538 patientpop
2018-08-05 21:13:07.933031 playfirst
2018-08-05 21:13:07.938558 audiencescience
2018-08-05 21:13:07.944155 tapad
2018-08-05 21:13:07.949709 mashery
2018-08-05 21:13:07.955201 vungle
2018-08-05 21:13:07.960696 udemy
2018-08-05 21:13:07.966200 domo
2018-08-05 21:13:07.971693 radiumone
2018-08-05 21:13:07.977181 imperva
2018-08-05 21:13:07.982690 meebo
2018-08-05 21:13:07.989220 pentaho
2018-08-05 21:13:07.994959 dailymotion
2018-08-05 21:13:08.000456 lifelock
2018-08-05 21:13:08.006201 indiegogo
2018-08-05 21:13:08.011682 sprinklr
2018-08-05 21:13:08.017623 livefyre
2018-08-05 21:13:08.023141 addthis
2018-08-05 21:13:08.028689 enernoc
2018-08-05 21:13:08.034190 extole
2018-08-05 21:13:08.039773 lifx
2018-08-05 21:13:08.045260 liveramp
2018-08-05 21:13:08.050879 marklogic
20

2018-08-05 21:13:09.392291 educents
2018-08-05 21:13:09.398079 tealium
2018-08-05 21:13:09.403580 demandware
2018-08-05 21:13:09.409078 carecloud
2018-08-05 21:13:09.414581 dataxu
2018-08-05 21:13:09.420079 zenoss
2018-08-05 21:13:09.425561 adaptly
2018-08-05 21:13:09.431064 tubemogul
2018-08-05 21:13:09.436551 aglocal
2018-08-05 21:13:09.442013 chartbeat
2018-08-05 21:13:09.447513 apperian
2018-08-05 21:13:09.453021 scribd
2018-08-05 21:13:09.458517 pixable
2018-08-05 21:13:09.463997 agilone
2018-08-05 21:13:09.469481 gainsight
2018-08-05 21:13:09.475014 avalara
2018-08-05 21:13:09.480528 younoodle
2018-08-05 21:13:09.486616 seesmic
2018-08-05 21:13:09.492101 shyp
2018-08-05 21:13:09.497576 mobileiron
2018-08-05 21:13:09.503091 kixeye
2018-08-05 21:13:09.508576 zulily
2018-08-05 21:13:09.514067 carepredict
2018-08-05 21:13:09.519552 kickstarter
2018-08-05 21:13:09.525055 spreadshirt
2018-08-05 21:13:09.530554 swrve
2018-08-05 21:13:09.536051 ciphercloud
2018-08-05 21:13:09.541524 inqu

2018-08-05 21:13:11.329505 trivago
2018-08-05 21:13:11.335221 beintoo
2018-08-05 21:13:11.340825 obopay
2018-08-05 21:13:11.346457 feedburner
2018-08-05 21:13:11.351960 tradesy
2018-08-05 21:13:11.357445 utilityapi
2018-08-05 21:13:11.362890 wrike
2018-08-05 21:13:11.368408 timehop
2018-08-05 21:13:11.373915 swifto
2018-08-05 21:13:11.379449 realmatch
2018-08-05 21:13:11.384949 newstore
2018-08-05 21:13:11.390478 inxpo
2018-08-05 21:13:11.395967 ensyn
2018-08-05 21:13:11.401480 nodejitsu
2018-08-05 21:13:11.407048 topix
2018-08-05 21:13:11.413041 junar
2018-08-05 21:13:11.418603 mzinga
2018-08-05 21:13:11.424096 rocana
2018-08-05 21:13:11.429578 abcmob
2018-08-05 21:13:11.435092 brightpearl
2018-08-05 21:13:11.440743 witricity
2018-08-05 21:13:11.446254 clypd
2018-08-05 21:13:11.451741 shopigniter
2018-08-05 21:13:11.457234 glamsquad
2018-08-05 21:13:11.462727 shopflick
2018-08-05 21:13:11.468207 tapru
2018-08-05 21:13:11.473709 peerapp
2018-08-05 21:13:11.479190 noom
2018-08-05 21:13:

...

2018-08-05 21:13:52.666924 adtriba
2018-08-05 21:13:52.671470 pressmatrix
2018-08-05 21:13:52.675978 sidestripe
2018-08-05 21:13:52.681158 xtuple
2018-08-05 21:13:52.685637 toposens
2018-08-05 21:13:52.690072 iforem
2018-08-05 21:13:52.694531 algenetix
2018-08-05 21:13:52.699016 lorious
2018-08-05 21:13:52.703437 lendamend
2018-08-05 21:13:52.707945 rvita
2018-08-05 21:13:52.712437 pathsensors
2018-08-05 21:13:52.717064 intelinair
2018-08-05 21:13:52.721745 continuent
2018-08-05 21:13:52.726432 egym
2018-08-05 21:13:52.731086 cargoguard
2018-08-05 21:13:52.735916 actacell
2018-08-05 21:13:52.740534 multivoice
2018-08-05 21:13:52.745698 classiphix
2018-08-05 21:13:52.750531 sicortex
2018-08-05 21:13:52.755166 givenext
2018-08-05 21:13:52.759763 wanamaker
2018-08-05 21:13:52.764381 freewayworks
2018-08-05 21:13:52.769211 glovico
2018-08-05 21:13:52.773985 maestroconference
2018-08-05 21:13:52.778631 metagenomix
2018-08-05 21:13:52.783260 makercraft
2018-08-05 21:13:52.788040 luxtech
2018

2018-08-05 21:13:54.027249 fyreball
2018-08-05 21:13:54.032356 lepow
2018-08-05 21:13:54.038084 hdmessaging
2018-08-05 21:13:54.043092 collegezen
2018-08-05 21:13:54.048118 badabroad
2018-08-05 21:13:54.053100 prisyna
2018-08-05 21:13:54.058140 dimensionalmechanics
2018-08-05 21:13:54.063105 putney
2018-08-05 21:13:54.068140 artaic
2018-08-05 21:13:54.073102 venuenext
2018-08-05 21:13:54.078159 apptrigger
2018-08-05 21:13:54.083194 seclarity
2018-08-05 21:13:54.088242 knowlent
2018-08-05 21:13:54.093208 sumomi
2018-08-05 21:13:54.098243 giiv
2018-08-05 21:13:54.103210 pylba
2018-08-05 21:13:54.108198 baremetrics
2018-08-05 21:13:54.113242 sochat
2018-08-05 21:13:54.118296 lockr
2018-08-05 21:13:54.123279 mobappcreator
2018-08-05 21:13:54.128268 ipling
2018-08-05 21:13:54.133237 biisafe
2018-08-05 21:13:54.138265 dewmobile
2018-08-05 21:13:54.143204 peerfit
2018-08-05 21:13:54.148179 greendimes
2018-08-05 21:13:54.153115 trackme
2018-08-05 21:13:54.158118 tendertree
2018-08-05 21:13:54.

2018-08-05 21:13:55.329273 dabkick
2018-08-05 21:13:55.333954 worksurfers
2018-08-05 21:13:55.338628 aimwith
2018-08-05 21:13:55.343237 igor
2018-08-05 21:13:55.347903 kluster
2018-08-05 21:13:55.352663 estorian
2018-08-05 21:13:55.357376 kashless
2018-08-05 21:13:55.361960 dermamedics
2018-08-05 21:13:55.366564 quantiacs
2018-08-05 21:13:55.371205 scalyr
2018-08-05 21:13:55.375805 trackin
2018-08-05 21:13:55.380429 classiqs
2018-08-05 21:13:55.385024 wireover
2018-08-05 21:13:55.389660 contix
2018-08-05 21:13:55.394363 moblico
2018-08-05 21:13:55.399012 healthentic
2018-08-05 21:13:55.403600 tempronics
2018-08-05 21:13:55.408247 cuisinelinks
2018-08-05 21:13:55.412841 weatherista
2018-08-05 21:13:55.418143 looklist
2018-08-05 21:13:55.422852 gozent
2018-08-05 21:13:55.427471 realops
2018-08-05 21:13:55.432095 bistrobot
2018-08-05 21:13:55.436700 conxtech
2018-08-05 21:13:55.441315 swingpal
2018-08-05 21:13:55.445931 ifingerlock
2018-08-05 21:13:55.450566 reachoo
2018-08-05 21:13:55.45

2018-08-05 21:13:56.706293 nextdigest
2018-08-05 21:13:56.710931 intrameta
2018-08-05 21:13:56.715522 liquiverse
2018-08-05 21:13:56.720157 vinepair
2018-08-05 21:13:56.725442 linksify
2018-08-05 21:13:56.730118 slidewinder
2018-08-05 21:13:56.734714 derbysoft
2018-08-05 21:13:56.739362 intruguard
2018-08-05 21:13:56.743961 readydock
2018-08-05 21:13:56.748579 septrx
2018-08-05 21:13:56.753201 pulmokine
2018-08-05 21:13:56.758100 hooja
2018-08-05 21:13:56.762739 baseclick
2018-08-05 21:13:56.767850 biodetego
2018-08-05 21:13:56.772560 socialthreader
2018-08-05 21:13:56.777250 findally


,uuid,description,kp_topicrank,kp_embedrank
company_name,,,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog...","[market, technical innovation, company, synthe...","[world class technology, natural tissue repair..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[classâ€ products, hemodialysis access graft,...","[human clinical trials, hemodialysis access gr..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[systems, development stage company, company, ...","[blood alcohol detection system, construction ..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...,"[thrombovision, medical diagnosis, biomedical ...","[medical diagnosis, critical cardiovascular ri..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[access policies, fortress capsule, manages, s...","[access policies, fortress capsule, virtual wo..."


In [7]:
# Save entire dataframe with keyphrases and confidences
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
with open(os.path.join(tempdir, 'temp_data', 'k_with_confidences_companies_df'), 'wb') as fp:
    pickle.dump(k_with_confidences_companies_df, fp)

In [8]:
# Save entire dataframe with keyphrases
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
with open(os.path.join(tempdir, 'temp_data', 'k_companies_df'), 'wb') as fp:
    pickle.dump(k_companies_df, fp)

In [5]:
with open (os.path.join(tempdir, 'temp_data', 'k_companies_df'), 'rb') as fp:
    k_companies_df = pickle.load(fp)
k_companies_list = k_companies_df.index.values

In [10]:
print(len(k_companies_df))
companies_list = companies_df.index.values
k_companies_df = k_companies_df[k_companies_df.index.isin(companies_list)]
print(len(k_companies_df))
k_companies_list = k_companies_df.index
print(len(k_companies_list))

k_companies_df.head()

9635
8599
8599


,uuid,description,kp_topicrank,kp_embedrank
company_name,,,,
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[classâ€ products, hemodialysis access graft,...","[human clinical trials, hemodialysis access gr..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[systems, development stage company, company, ...","[blood alcohol detection system, construction ..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...,"[thrombovision, medical diagnosis, biomedical ...","[medical diagnosis, critical cardiovascular ri..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[access policies, fortress capsule, manages, s...","[access policies, fortress capsule, virtual wo..."
songfor,7789da2f-f1dd-a1ac-434b-355c320b4d71,Auf SongFor.com hast du die Möglichkeit Musik ...,"[zu erleben, das einzige, auch immer, zum schl...","[sind deine lieblingssongs zum knutschen, emot..."


In [11]:
### Import the articles ###

In [12]:
# Get the list of paths to articles files and their sizes
paths = [(company, os.path.join(basedir, "News_1", "".join([company.upper(), ".json"]))) for company in k_companies_list]
paths = [path + (os.stat(path[1]).st_size,) for path in paths]

# Sort by sizes
paths = sorted(paths, key=lambda x: x[2], reverse=True)

full_size = 0
for f_size in paths:
    full_size = full_size + f_size[2]
print("appr. " + str(round(full_size/1024/1024/1024)) + " GB is the full size of our dataset")

appr. 18 GB is the full size of our dataset


In [ ]:
'''
from jsonstreamer import JSONStreamer 
streamer = JSONStreamer()
for index_f, path in enumerate(paths):
    with open(path[1]) as file: 
        for line in file: 
            streamer.consume(line)
'''

In [ ]:
'''
def load_json(filename):
    with open(filename, 'rb') as fd:
        parser = ijson.parse(fd)
        return parser
def objects(self,file):
    key = '-'
    for prefix, event, value in ijson.parse(file):
        if prefix == '' and event == 'map_key':  # found new object at the root
            key = value  # mark the key value
            builder = ObjectBuilder()
        elif prefix.startswith(key):  # while at this key, build the object
            builder.event(event, value)
            if prefix == key+".item" and event == 'end_map':  # found the end of an object at the current key, yield
                yield key, builder.value 
'''

In [19]:
# Choose the keyphrase approach
kp_approach = "kp_embedrank"

num_articles = 0
all_articles_dict = dict()
articles = list()

regex1 = re.compile('[%s]' % re.escape(string.punctuation))
regex2 = re.compile('[%s]' % re.escape("\n"))
regex3 = re.compile('[%s]' % re.escape(string.punctuation + "\n"))
#link_regex = re.compile('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+')
#email_regex = re.compile('[\w\.-]+@[\w\.-]+(\.[\w]+)+')

# Handling emails and links 
url_extractor = URLExtract()
tlds = [e[1:] for e in url_extractor._tlds]
space_regex = re.compile('\s*\.\s*(' + "|".join(tlds) + ")") # For deleting spaces around the dots in some urls

for index_f, path in enumerate(paths):
    #if 500 > index_f > 460:
        
    
    print(str(datetime.datetime.now()) + ' ' + path[0])   

    counter = 0

    with open(path[1], 'rb') as fd:
        for parsed_object in ijson.items(fd, "item"):

            counter = counter + 1
            #if counter > 3:
            #    break

            

            # Filter articles out
            try:
                article = parsed_object['content'].lower()
                article_dict = dict()
                num_articles = num_articles + 1
                
                # Check if the article is in English
                try:
                    if 'ENGLISH' not in set(parsed_object['language']):
                        continue
                except KeyError:
                    pass
                
                # Filter out articles with emails and weblinks    
                if not url_extractor.has_urls(space_regex.sub('.ru',parsed_object['content'])) and not url_extractor.has_urls(article):
                    pass
                else:
                    continue
                
                # Filter out articles without keywords extracted from companies descriptions
                if any(keyphrase in article for keyphrase in set(k_companies_df.loc[path[0], kp_approach])):
                    pass
                else:
                    continue
                
                paragraphs = article.split("\n\n")
                paragraphs = [regex2.sub(' ', paragraph) for paragraph in set(paragraphs)
                             if len(regex1.sub(' ', paragraph).split()) > 20 and path[0] in paragraph]
                if paragraphs:
                    article_dict["company"] = path[0]
                    article_dict["paragraphs"] = paragraphs
                    article_dict["full_text"] = article
                    articles.append(article_dict)
                
            except KeyError:
                continue



    fd.close()
# Save just in case
articles1 = articles


2018-08-30 01:04:51.642103 brightline
2018-08-30 01:05:46.160080 wherefor
2018-08-30 01:06:37.223941 musx
2018-08-30 01:07:57.197218 crono
2018-08-30 01:08:06.802746 gridco
2018-08-30 01:08:37.774193 lookback
2018-08-30 01:09:05.790460 enervault
2018-08-30 01:09:32.390535 energysavvy
2018-08-30 01:09:58.197029 humon
2018-08-30 01:10:00.997940 muut
2018-08-30 01:10:30.268402 bidgely
2018-08-30 01:10:51.908353 zqgame
2018-08-30 01:10:54.007992 fleetmatics
2018-08-30 01:11:53.454182 sungevity
2018-08-30 01:12:16.836825 determina
2018-08-30 01:12:50.007538 dealertrack
2018-08-30 01:13:46.403634 crowdstrike
2018-08-30 01:13:54.320060 declara
2018-08-30 01:14:31.803162 approva
2018-08-30 01:14:53.396335 motopia
2018-08-30 01:15:30.997232 stepout
2018-08-30 01:15:46.765626 plazes
2018-08-30 01:16:03.403970 upsnap
2018-08-30 01:16:06.501552 solfocus
2018-08-30 01:16:20.635884 everpower
2018-08-30 01:17:04.249318 akustica
2018-08-30 01:17:05.700029 tesaro
2018-08-30 01:17:16.999315 fullscreen
2

2018-08-30 01:39:29.122424 metacarta
2018-08-30 01:39:32.363582 quantcast
2018-08-30 01:39:36.081893 freshdesk
2018-08-30 01:39:40.336739 webscale
2018-08-30 01:39:43.158538 suniva
2018-08-30 01:39:49.972955 qualtrics
2018-08-30 01:39:54.476961 xirrus
2018-08-30 01:39:59.826364 kleer
2018-08-30 01:40:03.962967 kurtosys
2018-08-30 01:40:08.845028 klout
2018-08-30 01:40:13.450583 ignyta
2018-08-30 01:40:16.484790 otonomy
2018-08-30 01:40:19.978927 vovici
2018-08-30 01:40:24.205487 mirantis
2018-08-30 01:40:28.153154 hautelook
2018-08-30 01:40:34.172708 nanosys
2018-08-30 01:40:37.775500 eved
2018-08-30 01:40:41.463221 drik
2018-08-30 01:40:46.462492 orthosensor
2018-08-30 01:40:50.870544 speedconnect
2018-08-30 01:40:54.439510 mophie
2018-08-30 01:40:59.611919 dermira
2018-08-30 01:41:03.081799 blockboard
2018-08-30 01:41:05.320222 bigtincan
2018-08-30 01:41:15.192608 veracyte
2018-08-30 01:41:17.973232 hedgy
2018-08-30 01:41:22.832442 glycomimetics
2018-08-30 01:41:25.808940 cloudmark
2

2018-08-30 01:52:18.496394 spreadshirt
2018-08-30 01:52:19.943367 yub
2018-08-30 01:52:22.777957 taggle
2018-08-30 01:52:25.781616 duolingo
2018-08-30 01:52:28.176899 hoteltonight
2018-08-30 01:52:30.402517 worklight
2018-08-30 01:52:32.601356 getaround
2018-08-30 01:52:36.871699 fanduel
2018-08-30 01:52:39.320126 mapbox
2018-08-30 01:52:41.819834 sendgrid
2018-08-30 01:52:43.737790 iteam
2018-08-30 01:52:45.694351 urx
2018-08-30 01:52:47.860498 upek
2018-08-30 01:52:50.950834 tintri
2018-08-30 01:52:53.136760 novaled
2018-08-30 01:52:55.604580 eyelock
2018-08-30 01:52:58.179411 lilikoi
2018-08-30 01:53:00.068614 cerecor
2018-08-30 01:53:02.585586 decru
2018-08-30 01:53:05.728055 zenefits
2018-08-30 01:53:07.706723 fullstory
2018-08-30 01:53:10.606986 peerj
2018-08-30 01:53:12.784656 mixpanel
2018-08-30 01:53:13.919967 hipmunk
2018-08-30 01:53:16.049680 graphenea
2018-08-30 01:53:20.374326 qik
2018-08-30 01:53:22.646953 intela
2018-08-30 01:53:24.925011 greensky
2018-08-30 01:53:27.357

2018-08-30 01:59:55.444768 clarabridge
2018-08-30 01:59:56.799112 acumatica
2018-08-30 01:59:58.249214 precisionhawk
2018-08-30 01:59:59.827542 carecloud
2018-08-30 02:00:01.457594 cloudbees
2018-08-30 02:00:02.866405 mapmyfitness
2018-08-30 02:00:04.877208 invuity
2018-08-30 02:00:05.915479 infinio
2018-08-30 02:00:09.944170 comentis
2018-08-30 02:00:11.567746 novacea
2018-08-30 02:00:12.580147 socialtext
2018-08-30 02:00:14.245124 internetstores
2018-08-30 02:00:14.631348 drobo
2018-08-30 02:00:15.983367 bevi
2018-08-30 02:00:17.673950 vidyard
2018-08-30 02:00:19.361892 sipx
2018-08-30 02:00:21.088182 workproducts
2018-08-30 02:00:23.220317 safetraces
2018-08-30 02:00:24.722135 tradeking
2018-08-30 02:00:26.642584 backand
2018-08-30 02:00:28.644417 kymeta
2018-08-30 02:00:30.269627 playbuzz
2018-08-30 02:00:32.193113 zoodles
2018-08-30 02:00:33.409066 mediaocean
2018-08-30 02:00:35.011225 helpling
2018-08-30 02:00:35.908177 imaginab
2018-08-30 02:00:37.127079 flud
2018-08-30 02:00:38

2018-08-30 02:05:09.416475 bsafe
2018-08-30 02:05:10.619727 mashery
2018-08-30 02:05:11.508972 learnvest
2018-08-30 02:05:12.619672 collecta
2018-08-30 02:05:13.545591 bluechilli
2018-08-30 02:05:14.918600 solavei
2018-08-30 02:05:16.581918 paxvax
2018-08-30 02:05:17.611277 commonbond
2018-08-30 02:05:18.637875 knewton
2018-08-30 02:05:20.122333 livemocha
2018-08-30 02:05:21.195931 codenomicon
2018-08-30 02:05:22.492897 simplus
2018-08-30 02:05:23.343629 lycera
2018-08-30 02:05:24.406824 klocwork
2018-08-30 02:05:25.372212 tapad
2018-08-30 02:05:26.177293 socrata
2018-08-30 02:05:27.594307 fitocracy
2018-08-30 02:05:30.340506 mojio
2018-08-30 02:05:31.334143 humacyte
2018-08-30 02:05:32.337396 parstream
2018-08-30 02:05:33.427574 bluestacks
2018-08-30 02:05:34.530683 adjudica
2018-08-30 02:05:35.801765 vestigo
2018-08-30 02:05:37.161360 altheadx
2018-08-30 02:05:38.333353 yapta
2018-08-30 02:05:39.393488 ilypsa
2018-08-30 02:05:41.315151 wispry
2018-08-30 02:05:42.408787 conversocial
2

...

2018-08-30 02:32:52.816820 hashplex
2018-08-30 02:32:52.839895 paideasy
2018-08-30 02:32:52.850785 introfly
2018-08-30 02:32:52.853442 letsmake
2018-08-30 02:32:52.864947 baloonr
2018-08-30 02:32:52.881201 townhog
2018-08-30 02:32:52.898328 tellbird
2018-08-30 02:32:52.910137 vioozer
2018-08-30 02:32:52.920930 taltopia
2018-08-30 02:32:52.929643 nexhealth
2018-08-30 02:32:52.945060 changenexus
2018-08-30 02:32:52.958976 sitebrains
2018-08-30 02:32:52.974670 dynadec
2018-08-30 02:32:52.986194 viralninjas
2018-08-30 02:32:53.001638 beetmobile
2018-08-30 02:32:53.013733 kloudnation
2018-08-30 02:32:53.209187 mapistry
2018-08-30 02:32:53.233982 zursh
2018-08-30 02:32:53.247452 greenboxny
2018-08-30 02:32:53.264228 zootrock
2018-08-30 02:32:53.279274 careerlark
2018-08-30 02:32:53.296632 musistic
2018-08-30 02:32:53.307015 hunite
2018-08-30 02:32:53.316866 plumzi
2018-08-30 02:32:53.330340 futurederm
2018-08-30 02:32:53.345323 envirokure
2018-08-30 02:32:53.360987 accuitis
2018-08-30 02:32:

2018-08-30 02:32:59.943207 discoverly
2018-08-30 02:32:59.947637 worldhistoryproject
2018-08-30 02:32:59.972149 rapidrabbit
2018-08-30 02:32:59.985743 wemonitor
2018-08-30 02:32:59.996790 netccm
2018-08-30 02:33:00.004674 farmivore
2018-08-30 02:33:00.018456 cityspade
2018-08-30 02:33:00.028151 mywerx
2018-08-30 02:33:00.042638 looksima
2018-08-30 02:33:00.056326 ignitefeedback
2018-08-30 02:33:00.068381 weplann
2018-08-30 02:33:00.078331 airfordable
2018-08-30 02:33:00.389768 enovalys
2018-08-30 02:33:00.413561 videopixie
2018-08-30 02:33:00.486353 onovative
2018-08-30 02:33:00.502427 schoolinks
2018-08-30 02:33:00.513462 lumatix
2018-08-30 02:33:00.521945 phorent
2018-08-30 02:33:00.536287 shopalytic
2018-08-30 02:33:00.547712 ranktab
2018-08-30 02:33:00.559300 prizmiq
2018-08-30 02:33:00.569126 usercycle
2018-08-30 02:33:00.810005 daogames
2018-08-30 02:33:00.820813 metabrite
2018-08-30 02:33:00.836540 clusterflunk
2018-08-30 02:33:00.852816 monetsu
2018-08-30 02:33:00.864233 puzzli

2018-08-30 02:33:06.612257 streetowl
2018-08-30 02:33:06.635899 viirt
2018-08-30 02:33:06.645944 chatlingual
2018-08-30 02:33:06.661602 ekonnekt
2018-08-30 02:33:06.669733 fanbread
2018-08-30 02:33:06.677304 roomations
2018-08-30 02:33:06.684805 sumomi
2018-08-30 02:33:06.694185 gigzolo
2018-08-30 02:33:06.703136 teneology
2018-08-30 02:33:06.713573 pocketsuite
2018-08-30 02:33:06.726716 snowvation
2018-08-30 02:33:06.734306 pingpad
2018-08-30 02:33:06.742095 endoinsight
2018-08-30 02:33:06.749014 venuefox
2018-08-30 02:33:06.760416 mogotix
2018-08-30 02:33:06.770971 optoatmospherics
2018-08-30 02:33:06.778540 retailmls
2018-08-30 02:33:06.957424 cartnav
2018-08-30 02:33:06.967474 lawkick
2018-08-30 02:33:06.977962 juiceqube
2018-08-30 02:33:06.983244 binaryvr
2018-08-30 02:33:06.992160 iperlane
2018-08-30 02:33:07.001705 appslingr
2018-08-30 02:33:07.016399 wearpoint
2018-08-30 02:33:07.030469 cureeo
2018-08-30 02:33:07.044825 windensity
2018-08-30 02:33:07.058862 conjecta
2018-08-30 

2018-08-30 02:33:09.209646 stiqrd
2018-08-30 02:33:09.210662 scorebeyond
2018-08-30 02:33:09.211341 custex
2018-08-30 02:33:09.211990 pediaq
2018-08-30 02:33:09.212638 appmyday
2018-08-30 02:33:09.213284 ordrx
2018-08-30 02:33:09.213926 orbose
2018-08-30 02:33:09.214556 fanzter
2018-08-30 02:33:09.215182 orderbolt
2018-08-30 02:33:09.215806 offersavvy
2018-08-30 02:33:09.216428 crowdclock
2018-08-30 02:33:09.217067 myworldwall
2018-08-30 02:33:09.217740 revolucionadolabs
2018-08-30 02:33:09.218385 deliveryedge
2018-08-30 02:33:09.219032 mobileigniter
2018-08-30 02:33:09.219686 nextinit
2018-08-30 02:33:09.220334 ligandal
2018-08-30 02:33:09.220975 selligy
2018-08-30 02:33:09.221675 gripeo
2018-08-30 02:33:09.222328 chronowake
2018-08-30 02:33:09.222977 casahop
2018-08-30 02:33:09.223617 takokat
2018-08-30 02:33:09.224237 adextent
2018-08-30 02:33:09.224881 zairge
2018-08-30 02:33:09.225523 bottlestonight
2018-08-30 02:33:09.226175 coincube
2018-08-30 02:33:09.226818 gabbox
2018-08-30 0

2018-08-30 02:33:09.892011 atmosferiq
2018-08-30 02:33:09.894691 zoopshop
2018-08-30 02:33:09.895624 gigturn
2018-08-30 02:33:09.896289 smackages
2018-08-30 02:33:09.896938 spinthecam
2018-08-30 02:33:09.897642 gvmachines
2018-08-30 02:33:09.898391 compblue
2018-08-30 02:33:09.899139 ibabybox
2018-08-30 02:33:09.899884 worksurfers
2018-08-30 02:33:09.900600 gozent
2018-08-30 02:33:09.901331 swingpal
2018-08-30 02:33:09.902099 entrecard
2018-08-30 02:33:09.902841 traceair
2018-08-30 02:33:09.903593 ubrlocal
2018-08-30 02:33:09.904297 strongintro
2018-08-30 02:33:09.904951 livarava
2018-08-30 02:33:09.905627 upurskill
2018-08-30 02:33:09.906242 zenprospect
2018-08-30 02:33:09.906868 acrinta
2018-08-30 02:33:09.907495 bloomojo
2018-08-30 02:33:09.908141 chorafarma
2018-08-30 02:33:09.908789 vixxenn
2018-08-30 02:33:09.909425 boxventures
2018-08-30 02:33:09.910067 idlinker
2018-08-30 02:33:09.910711 cocollage
2018-08-30 02:33:09.911366 studiocracy
2018-08-30 02:33:09.912010 agentrave
2018-

In [ ]:
# Old code for json library, too slow as it reads every file into RAM
'''
for index_f, file in enumerate(paths):
    #if index_f > 0:
    #    break
    
    print(file)
    print(index_f)

    try:
        articles = pd.read_json(file[1])
    except ValueError:
        print("empty/faulty file")
        continue

    if "content" not in articles.columns:
        print("no content")
        continue

    articles.loc[:, "company"] = file[0]
    articles.loc[:, "uuid"] = k_company_df.loc[file[0].lower(), "uuid"]
    articles.loc[:, "description"] = k_company_df.loc[file[0].lower(), "description"]

    articles.dropna(subset=["content"], inplace=True)

    texts = [regex.sub('', i).lower() for i in articles.loc[:, "content"]]
    keyphrases = ast.literal_eval(k_company_df.loc[file[0].lower(), "kp_topicrank"])
    num_articles = num_articles + len(texts)

    # Check whether the any of the keywords exist in the text
    bi = [1 
             if any([keyphrase in text 
                     for (keyphrase, prob) in (keyphrases)]) and 250 < len(str.split(text)) < 500 
             else 0
            for text in texts]

    articles.loc[:, "kp_topicrank_bi"] = bi
    articles.dropna(subset=["content"], inplace=True)

    all_articles = all_articles.append(articles, ignore_index=True, sort=False)
    
''' 

In [20]:
# Save entire list of dicts with articles
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
with open(os.path.join(tempdir, 'temp_data', 'articles'), 'wb') as fp:
    pickle.dump(articles, fp)

In [21]:
with open (os.path.join(tempdir, 'temp_data', 'articles'), 'rb') as fp:
    articles = pickle.load(fp)
    

In [254]:
# Takes too long to delete duplicates in a nested list
'''
tmp=list()
encountered=list()
articles = articles1
for i in articles:
    if i["content"] not in encountered:
        encountered.append(i["content"])
        tmp.append(i)
num_dup = len(articles)-len(tmp)
print(str(num_dup) + " articles are duplicates")
articles = tmp
len(tmp)
'''



True

In [22]:
articles_df = pd.DataFrame(articles)
articles_df["paragraphs"] = articles_df['paragraphs'].apply(lambda x: '\n\n '.join(x))
articles_df.drop_duplicates(keep="first", subset="full_text", inplace=True)
num_dup = len(articles)-len(articles_df)
print(str(num_dup) + " articles are duplicates")

6210 articles are duplicates


In [23]:
print(str(len(articles_df)/(num_articles-num_dup)*100) + "% of articles (" +
      str(len(articles_df)) + " out of " + str(num_articles-num_dup) +
      ") contain corresponding keyphrases extracted from the descriptions of the companies, the name of the company in the relevant paragraphs, but do not contain email or web addresses")

4.589284654360226% of articles (59535 out of 1297261) contain corresponding keyphrases extracted from the descriptions of the companies, the name of the company in the relevant paragraphs, but do not contain email or web addresses


In [24]:
# Save dataframe with articles
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
with open(os.path.join(tempdir, 'temp_data', 'articles_df'), 'wb') as fp:
    pickle.dump(articles_df, fp)

In [25]:
with open (os.path.join(tempdir, 'temp_data', 'articles_df'), 'rb') as fp:
    articles_df = pickle.load(fp)
articles_df.head()

,company,full_text,paragraphs
0,brightline,plus company updates(pcu)\n\n ...,new york: following is the company profile of...
1,brightline,plus company updates(pcu)\n\n ...,"""media agencies, creative agencies and publi..."
2,brightline,m2 equitybites\n\n ...,brightline is the market leader for advanced t...
3,brightline,plus company updates(pcu)\n\n ...,austin: sizmek inc. has issued the following ...
4,brightline,broadcasting and cable\n\n ...,naylor said the majority of hulu's viewing now...


In [215]:
print(json.dumps(articles[0:10], sort_keys=True, indent=4))

[
    {
        "company": "brightline",
        "content": [
            " new york: following is the company profile of brightline gp, llc: brightline was the first company to pioneer dynamic television ad solutions for brands. today brightline offers incast, the industry's most powerful advanced tv technology suite that gives marketers a plug-and-play solution  to take advantage of the entire scale of over-the-top and smart tv advertising opportunities. build once. deploy anywhere. our ad product suite transforms passive content into dynamic, deep, interactive experiences  that work seamlessly across the connected tv universe. our server delivers truly universal ad experiences to all ott devices - automatically optimizing  for unique technology and user behavior along the way."
        ]
    },
    {
        "company": "brightline",
        "content": [
            " austin: sizmek inc. has issued the following press release:  sizmek, the open ad management company for multiscreen a